

<h1 style="font-size:35px;color:orange"><center>SENTIMENT ANALYSIS <center></h2>

In [1]:
#Importing all required library
import nltk
import re
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

In [65]:
Dataset = pd.read_csv('Dataset.csv')

In [66]:
Dataset.head()

,Article,label
0,you can smell hillary s feardaniel greenfield ...,0
1,watch the exact moment paul ryan committed pol...,0
2,kerry to go to paris in gesture of sympathyu s...,1
3,bernie supporters on twitter erupt in anger ag...,0
4,the battle of new york why this primary matte...,1


In [51]:
text=Dataset["Article"]

<h2 style="color:orange;"> TF-IDF VECTORIZER</h2>

In [52]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(Dataset['Article'] )
tfidf

<76480x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 23587211 stored elements in Compressed Sparse Row format>


<h2 style="color:orange;">Sentiment analysis using VADER</h2>

In [53]:
sentiment_analyzer = VS()
def count_tags(tweet_c):  
    
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', tweet_c)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def sentiment_analysis(tweet):   
    sentiment = sentiment_analyzer.polarity_scores(tweet)    
    twitter_objs = count_tags(tweet)
    features = [sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],twitter_objs[0], twitter_objs[1],
                twitter_objs[2]]
    #features = pandas.DataFrame(features)
    return features

def sentiment_analysis_array(tweets):
    features=[]
    for t in tweets:
        features.append(sentiment_analysis(t))
    return np.array(features)

final_features = sentiment_analysis_array(text)
final_features

new_features = pd.DataFrame({'Neg':final_features[:,0],'Pos':final_features[:,1],'Neu':final_features[:,2],'Compound':final_features[:,3],
                            'url_tag':final_features[:,4],'mention_tag':final_features[:,5],'hash_tag':final_features[:,6]})
new_features

,Neg,Pos,Neu,Compound,url_tag,mention_tag,hash_tag
0,0.184,0.088,0.728,-0.9995,0.0,0.0,0.0
1,0.081,0.090,0.829,0.5119,0.0,0.0,0.0
2,0.062,0.092,0.846,0.9382,0.0,0.0,0.0
3,0.089,0.090,0.821,-0.1027,0.0,0.0,0.0
4,0.084,0.179,0.737,0.9917,0.0,0.0,0.0
...,...,...,...,...,...,...,...
76475,0.163,0.142,0.694,0.1027,0.0,0.0,0.0
76476,0.055,0.149,0.796,0.9884,0.0,0.0,0.0
76477,0.000,0.000,1.000,0.0000,0.0,0.0,0.0
76478,0.018,0.158,0.824,0.9916,0.0,0.0,0.0


In [54]:
sentiment_category_tb = ['positive' if score > 0 
                             else 'negative' if score < 0 
                                 else 'neutral' 
                                     for score in new_features.Compound]

In [57]:
df = pd.DataFrame([list(Dataset['Article']), new_features.Compound, sentiment_category_tb]).T

In [ ]:
# df.columns = ['Article', 'sentiment_score', 'sentiment_category']
# df['sentiment_score'] = df.sentiment_score.astype('float')
# df.groupby(by=['Article']).describe()

In [69]:
Dataset["label"]=sentiment_category_tb 

In [70]:
Dataset.head()

,Article,label
0,you can smell hillary s feardaniel greenfield ...,negative
1,watch the exact moment paul ryan committed pol...,positive
2,kerry to go to paris in gesture of sympathyu s...,positive
3,bernie supporters on twitter erupt in anger ag...,negative
4,the battle of new york why this primary matte...,positive


In [71]:
Dataset['label'].value_counts()

positive    38279
negative    36704
neutral      1497
Name: label, dtype: int64